<h1 align=center><font size = 5>SPE Europe Energy GeoHackathon</font></h1>

Society of Petroleum Engieers|  ERCE Equipoise | Geothermal Challenge  
:-------------------------:|:-------------------------:|:-------------------------:
<a href="https://www.spehackathon-eu.com"> <img src = "https://upload.wikimedia.org/wikipedia/en/thumb/4/42/Society_of_Petroleum_Engineers_Logo.svg/1200px-Society_of_Petroleum_Engineers_Logo.svg.png" width = 200 /> | <a href="https://www.erce.energy/"> <img src = "https://energycouncil.com/wp-content/uploads/ERCE.png" width = 200/> | <img src = "https://solutions.vallourec.com/-/media/Solutions-img/Low-Carbon-Energy/Medias/Geothermal-energy/Header-Corner_Geothermal.ashx" width = 200/>

<h1 align=center><font size = 4> Team ERCE - Munish Kumar, Kanna Swaminathan & Ivan Maulana </font></h1>
<h1 align=center><font size = 3> 2021 </font></h1>

Code here generates a fairway map from static properties derived from the well logs and well test

Import Libraries

In [1]:
# Uncomment if needed
#!pip install dash

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
import plotly
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator
from itertools import cycle
#import dash
#from dash import dcc
#from dash import html

Load Data

In [3]:
df = pd.read_csv('Wells.csv')
df

,Unnamed: 0,Label,Type,x,y,Cluster,dist_D1,dist_D2,phi,k,Net Thickness
0,12,D1,Demand,94,65,0,0.000000,109.480592,NaN,NaN,NaN
1,7,I02,Injector,136,75,0,43.174066,95.634722,0.1990,1108.824338,27.33600
2,8,I03,Injector,109,113,0,50.289164,59.203040,0.1518,575.398382,152.26624
3,2,E03,Exploration,46,46,0,51.623638,147.027208,0.1786,849.327413,200.64000
4,3,E04,Exploration,52,100,0,54.671748,101.138519,NaN,646.327975,NaN
5,1,E02,Exploration,145,41,0,56.364883,130.541181,NaN,751.443521,NaN
6,10,P02,Producer,162,87,0,71.470274,90.873538,NaN,NaN,NaN
7,0,E01,Exploration,195,70,0,101.123687,122.065556,0.1487,548.123345,46.65800
8,11,P03,Producer,134,115,1,64.031242,55.731499,NaN,NaN,NaN
9,9,P01,Producer,164,110,1,83.216585,71.561163,0.1680,732.476238,67.79120


Normalise data

In [4]:
#normalisation of k and net thickness to P01
df['k'] = (df['k'] - df['k'].iloc[9]) / (df['k'].max() - df['k'].min())
df['Net Thickness'] = (df['Net Thickness'] - df['Net Thickness'].iloc[9]) / (df['Net Thickness'].max() - df['Net Thickness'].min())

#create feature of spatial variation of permeability relative to P01
#df['k'] =  df[['phi','k_norm']].sum(axis=1, min_count=1)
df

,Unnamed: 0,Label,Type,x,y,Cluster,dist_D1,dist_D2,phi,k,Net Thickness
0,12,D1,Demand,94,65,0,0.000000,109.480592,NaN,NaN,NaN
1,7,I02,Injector,136,75,0,43.174066,95.634722,0.1990,0.600821,-0.233435
2,8,I03,Injector,109,113,0,50.289164,59.203040,0.1518,-0.250767,0.487438
3,2,E03,Exploration,46,46,0,51.623638,147.027208,0.1786,0.186547,0.766565
4,3,E04,Exploration,52,100,0,54.671748,101.138519,NaN,-0.137531,NaN
5,1,E02,Exploration,145,41,0,56.364883,130.541181,NaN,0.030280,NaN
6,10,P02,Producer,162,87,0,71.470274,90.873538,NaN,NaN,NaN
7,0,E01,Exploration,195,70,0,101.123687,122.065556,0.1487,-0.294310,-0.121943
8,11,P03,Producer,134,115,1,64.031242,55.731499,NaN,NaN,NaN
9,9,P01,Producer,164,110,1,83.216585,71.561163,0.1680,0.000000,0.000000


Plot spatial variation of permeability relative to P01

In [12]:
#colors = plotly.colors.qualitative.swatches()
#colors.show()
colors = cycle(plotly.colors.qualitative.Set1)

fig1 = go.Figure()
fig1.add_trace(go.Contour(
                x = df['x'],
                y = df['y'],
                z = df['k']))

for s in df.Type.unique():
    dfi = df[df.Type == s]
    fig1.add_trace(go.Scatter(x = dfi['x'],
                              y = dfi['y'],
                              mode = 'markers',
                              name = s,
                              marker = dict(size=16),
                              marker_color = next(colors),
                              customdata = np.stack((dfi['Label'], dfi['k'], dfi['Type']), axis = -1),
                              hovertemplate =
                              '<b>Location Name:</b> %{customdata[0]}' +
                              '<br><b>Perm. relative to P01:</b> %{customdata[1]:.2f}' +
                              '<br><b>Type:</b> %{customdata[2]}<extra></extra>',
                              #autocontour = True
                             ))

fig1.update_layout(legend_orientation="h", xaxis_range = [0,230], yaxis_range = [0,206])    
    
fig1.show()

Plot spatial variation of net thickness relative to P01

In [13]:
colors = cycle(plotly.colors.qualitative.Set1)

fig2 = go.Figure()
fig2.add_trace(go.Contour(
                x = df['x'],
                y = df['y'],
                z = df['Net Thickness']))

for s in df.Type.unique():
    dfi = df[df.Type == s]
    fig2.add_trace(go.Scatter(x = dfi['x'],
                              y = dfi['y'],
                              mode = 'markers',
                              name = s,
                              marker = dict(size=16),
                              marker_color = next(colors),
                              customdata = np.stack((dfi['Label'], dfi['Net Thickness']), axis = -1),
                              hovertemplate =
                              '<b>Well Name:</b> %{customdata[0]}' +
                              '<br><b>Net thickness relative to P01:</b> %{customdata[1]:.2f}<extra></extra>'
                             ))

fig2.update_layout(legend_orientation="h", xaxis_range = [0,230], yaxis_range = [0,206])    
    
fig2.show()

The southwest quadrant is high-graded as the area with higher permeability and higher net thickness according to the static properties we calculated from the data.